# Project ETL part 2

The objective is to process two datasets, transform and merge them to make a report to the warehouse operations

In [ ]:
import com.amazonaws.auth.{AWSStaticCredentialsProvider, BasicAWSCredentials}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import org.apache.spark.sql.expressions.Window

In [ ]:
val awsAccessKeyId = "**********"
val awsSecretKey = "***********"
val kinesisStreamName = "Jedha-pickers"
val kinesisRegion = "eu-west-1" 

We set the aws environement variables on the spark cluster

In [ ]:
sc.hadoopConfiguration.set("fs.s3a.access.key", awsAccessKeyId)
sc.hadoopConfiguration.set("fs.s3a.secret.key", awsSecretKey)

We read the created by Firehose

In [ ]:
var orders = spark.read.json("s3://path/defined/in/firehose/*/*/*/*/*")

We parse the date as a timestamp

In [ ]:
orders = orders.withColumn("timestamp", date_format($"timestamp","yyyy-MM-dd HH-mm-ss.mm"))

For each order we only select the latest one

In [ ]:
val w = Window.partitionBy($"order_id").orderBy($"timestamp".desc)
val only_latest_orders = orders.withColumn("rank", row_number().over(w)).where($"rank" === 1).drop("rank")

This is very condensed:
- On the first line we parse the array of items
- One the 2nd line we extract the block form the location string
- On the 3d line we select the nested sku
- Then we remove the temporary column "block" Groupy by block, and count the items for each block.

In [ ]:
val itemsPerBlock = only_latest_orders
      .select($"order_id", $"status", $"timestamp", explode($"items").as("item"))
      .withColumn("block", regexp_extract($"item.location", "BLOCK-(.+)-(.+)", 2))
      .withColumn("sku", $"item.sku")
      .drop("item")
      .groupBy($"block")
      .count()
      .as("items_to_pick")

We use spark streaming to get the pickers

In [ ]:
val kinesis = spark.readStream
  .format("kinesis")
  .option("streamName", kinesisStreamName)
  .option("region", kinesisRegion)
  .option("initialPosition", "TRIM_HORIZON")
  .option("checkpointLocation", "s3://output-databricks/checkpoint/")
  .option("awsAccessKey", awsAccessKeyId)
  .option("awsSecretKey", awsSecretKey)
  .load()


val schema = StructType(Seq(
  StructField("block", StringType),
  StructField("action", StringType),
))

val result = kinesis.selectExpr("CAST(data as STRING) as json")
  .withColumn("picker", from_json($"json", schema))
  .withColumn("block", expr("picker.block"))
  .withColumn("action", expr("picker.action"))
  .withColumn("picker_delta", when(($"action" === "add"), 1).otherwise(-1))
  .drop("json")
  .drop("action")
  .drop("order")
  .groupBy("block").agg(sum($"picker_delta").as("pickers"))
  .withColumn("items_per_hour", $"pickers" * 10)
  .drop("pickers")
  .join(itemsPerBlock, "block")


Nice to have on databricks: 
you can make a plot to visualize the blocks. 
Use:
- **keys**: `block`
- **values**: `count`, `items_per_hour`

In [ ]:
display(result)